In [15]:
%pip install python-dotenv langchain langchain_openai langchain_community tiktoken scikit-learn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 100.7 kB/s eta 0:00:00B/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 9.6 MB/s eta 0:00:000m eta 0:00:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext dotenv
%dotenv
#  add a .env file to the repo with OPENAI_API_KEY=<your open ai key>

In [3]:
import re

with open('data/PGEssays/mit.txt') as file:
    essay = file.read()

single_sentences_list = re.split(r'(?<=[(\.|\n\n)?!])\s+', essay)
print (f"{len(single_sentences_list)} senteneces were found")

335 senteneces were found


In [4]:
def clean_sentence(x):
    return x.replace('\n', ' ') + ("." if not x.endswith('.') and not x.endswith('?') and not x.endswith('!') else '')

sentences = [{'sentence': clean_sentence(x), 'index' : i} for i, x in enumerate(single_sentences_list)]

In [5]:
def combine_sentences(sentences, buffer_size=1):
    for i in range(len(sentences)):
        combined_sentence = ''
    
        # Add sentences before the current one, based on the buffer size.
        for j in range(i - buffer_size, i):
            if j >= 0:
                combined_sentence += sentences[j]['sentence'] + ' '
        
        # Add the current sentence
        combined_sentence += sentences[i]['sentence']

        for j in range(i + 1, i + 1 + buffer_size):
            if j < len(sentences):
                combined_sentence += ' ' + sentences[j]['sentence']
                
        sentences[i]['combined_sentence'] = combined_sentence

    return sentences

sentences = combine_sentences(sentences, buffer_size = 2)

In [16]:
from langchain_openai.embeddings import OpenAIEmbeddings
import os
import numpy as np
oaiembeds = OpenAIEmbeddings( api_key = os.getenv('OPENAI_API_KEY'))

In [17]:
embeddings = oaiembeds.embed_documents([x['combined_sentence'] for x in sentences])

In [18]:
from sklearn.cluster import HDBSCAN

def test_clustering_model(model, data):
    labels = model.fit_predict(data)
    grouped_sentences = {
    l : [] for l in set(labels) 
    }
    for i,sentence in enumerate(sentences):
        grouped_sentences[labels[i]] += [sentence['sentence']]
    return labels, grouped_sentences

In [113]:
labels, grouped_sentences = test_clustering_model(HDBSCAN(), embeddings)
labels

array([-1, -1, -1, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 13, -1, 13, -1, 13, 13, -1, 13, -1,
        9,  9, -1,  9,  9,  9,  9,  9, -1, -1, -1, -1, -1,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6, -1,  6,  6,  6,  6,  6,  6,  6,
       -1,  6, -1,  6, -1,  7,  7,  7,  7,  7,  1,  1,  1,  1,  1,  1,  1,
       -1, -1, -1, -1, -1,  4,  4,  4, -1,  4,  4,  4, -1,  4,  4, -1,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3, -1,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5, -1, -1, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, -1, 18, -1, -1, 19, 19, -1, 19, -1, 19, -1, 19,
       19, -1, -1, -1, 16, 16, -1, 16, -1, 16, -1, 16, 16, -1, 16, 16, 16,
       16, 16, 16, 16, -1, 16, 16, 16, 16, 16, -1, 11, 11, 11, 11, 11, 11,
       11, 11, 11, -1, -1, -1, -1, -1, -1,  0, -1,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [114]:
def merge_outliers(labels):
    index_ranges = []
    curr_num = [l for l in labels if l != -1][0]
    curr_entry = {"start" : 0, "value" : curr_num}
    for i,l in enumerate(labels): 
        if l == -1: continue
        if curr_num != l:
            curr_entry['end'] = i-1
            index_ranges.append(curr_entry)
            curr_entry = {"start" : i, "value":l}
            curr_num = l
    curr_entry['end'] = i
    index_ranges.append(curr_entry)
    groups = []
    for i,l in enumerate(labels): 
        if l == -1:
            nearest = -1
            nearest_dist = len(labels)
            for n in index_ranges:
                if i < n['end'] and i > n['start']:
                    nearest = n['value']
                    break
                closest_side = min(abs(i - n['end']), abs(i - n['start']))
                if closest_side < nearest_dist:
                    nearest_dist = closest_side
                    nearest = n['value']
            groups.append(nearest)
    labels_fixed = [ ]
    outlier_counter = 0
    for l in labels:
        if l == -1:
            labels_fixed.append(groups[outlier_counter])
            outlier_counter+=1
        else:
            labels_fixed.append(l)
    labels_fixed  = np.array(labels_fixed)
    grouped_sentences = {
    l : [] for l in set(labels_fixed) 
    }
    for i,sentence in enumerate(sentences):
        grouped_sentences[labels_fixed[i]] += [sentence['sentence']]
    return labels_fixed, grouped_sentences
labels_fixed, grouped_sentences = merge_outliers(labels)
labels_fixed

array([12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [21]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier().fit(embeddings,labels_fixed)

In [132]:
from collections import defaultdict
def get_extracts(model, questions):
    questions_embed = oaiembeds.embed_documents(questions)
    # get K closest nodes and their distances
    distances, indices = model.kneighbors(questions_embed)
    output = {}
    for i, q in enumerate(questions):
        
        pivot_map = defaultdict(list)
        # get the clusters assigned for each closest node
        clusters = labels_fixed[indices[i]]

        for key, value in zip(clusters, distances[i]):
            pivot_map[key].append(value)
        output[q] = [ { "% nearest neighbours match" : len(l) / len(distances[0]), 
                              "text" : " ".join(grouped_sentences[k]), 
                              "avg distance":sum(l)/len(l),
                             "cluster" : k,
                              "min distance" : min(l),
                              "max distance" : max(l)
                             } for k, l in pivot_map.items()]
    return output

In [133]:
get_extracts(model, ["how to make money fast", "work ethic", "toyota yeti"])

{'how to make money fast': [{'% nearest neighbours match': 0.4,
   'text': 'One\'s first thought tends to be simply "This sucks. I\'m in debt. Plus I have to get up on monday and go to work."  Gradually you realize that these two things are as tightly connected as only a market can make them.So the most important advantage 24 year old founders have over 20 year old founders is that they know what they\'re trying to avoid. To the average undergrad the idea of getting rich translates into buying Ferraris, or being admired. To someone who has learned from experience about the relationship between money and work, it translates to something way more important: it means you get to opt out of the brutal equation that governs the lives of 99.9% of people. Getting rich means you can stop treading water.Someone who gets this will work much harder at making a startup succeed—with the proverbial energy of a drowning man, in fact. But understanding the relationship between money and work also chang

In [120]:

get_extracts(model, ["What can 25 year olds do that 32 year olds can't"])

{"What can 25 year olds do that 32 year olds can't": [(0.8,
   "Maybe the window will close on some idea you're working on, but that won't be the last idea you'll have. For every idea that times out, new ones become feasible. Historically the opportunities to start startups have only increased with time.In that case, you might ask, why not wait longer? Why not go work for a while, or go to grad school, and then start a startup? And indeed, that might be a good idea. If I had to pick the sweet spot for startup founders, based on who we're most excited to see applications from, I'd say it's probably the mid-twenties. Why? What advantages does someone in their mid-twenties have over someone who's 21? And why isn't it older? What can 25 year olds do that 32 year olds can't? Those turn out to be questions worth examining.PlusIf you start a startup soon after college, you'll be a young founder by present standards, so you should know what the relative advantages of young founders are. They'r

In [30]:
model.kneighbors(question)

(array([[0.51221136, 0.5207131 , 0.52904578, 0.53697799, 0.53935413]]),
 array([[58, 56, 55, 57, 74]]))

In [32]:
model.predict_proba(question)

array([[0. , 0. , 0. , 0. , 0. , 0. , 0.8, 0.2, 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. ]])

In [33]:
dists, inds = model.kneighbors(question)

In [35]:
labels_fixed[inds]

array([[6, 6, 6, 6, 7]])

In [69]:
question = oaiembeds.embed_documents([''])

get_extracts(model, question)

[(1.0,
  'that we came out with that was really great, I\'d never once done . When you know nothing, you have to reinvent stuff for yourself, and if you\'re smart your reinventions may be better than what preceded them. This is especially true in fields where the rules change. All our ideas about software were developed in a time when processors were slow, and memories and disks were tiny. Who knows what obsolete assumptions are embedded in the conventional wisdom? And the way these assumptions are going to get fixed is not by explicitly deallocating them, but by something more akin to garbage collection. Someone ignorant but smart will come along and reinvent everything, and in the process simply fail to reproduce certain existing ideas.MinusSo much for the advantages of young founders. What about the disadvantages? I\'m going to start with what goes wrong and try to trace it back to the root causes.What goes wrong with young founders is that they build stuff that looks like class pro

In [119]:
grouped_sentences.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])